In [58]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
import os

#Variables initialisation

src = 'данные SiO2 на нескольких частотах'
out = 'result'
table_name = 'данные SiO2 на нескольких частотах/ps+20% SiO2.xlsx'
list_name = 'Лист1'

In [54]:
wb = openpyxl.load_workbook(filename = table_name)
sheet = wb[list_name]

In [55]:
sheet.cell(row=1, column=1)

<Cell 'Лист1'.A1>

In [68]:
freq_group = 0          # номер частоты, данные для которого обрабатываются (с нуля)

for freq_group in range(6):
    column_s = freq_group*4 + 1
    freq_str = sheet.cell(row=1, column=column_s+2).value
    if not freq_str.endswith('Мгц') :
        print(f"WARNING!!! Unexpected frequance format in row 1, column {column_s+2}! {freq_str} doesn't ends with 'Мгц'")
    freq = float((freq_str[:-3]).strip().replace(',', '.'))      # in Mhz 
    
    dirname = os.path.join(out, freq_str)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    
    axis_name = ['z','y','x']
    naming_row = [2, 22, 42]

    for direction in [0, 1, 2]:         # условная нумерация, с определением через несколько списков
        row_s = naming_row[direction]+1
        row_end = row_s
        while sheet.cell(row=row_end, column=column_s).value is not None:
            row_end+=1
        df = pd.DataFrame(
                {sheet.cell(row=naming_row[direction], column=column_s).value: 
                  [i for i, in sheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=column_s, max_col=column_s,  values_only=True)], 
                 'file_name': 
                  ['WFM' + (str(i) if len(str(i)) > 1 else '0'+str(i))+'.csv' for i, in 
                       sheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=column_s+1, max_col=column_s+1,  values_only=True)]
                }
            )
        df.to_csv(os.path.join(dirname, axis_name[direction]+'.csv'))

In [41]:
[i.value for i,  in sheet.iter_rows(min_row=1, max_row=10, min_col=2, max_col=2)]


['PS + 20% Sio2', 'номер данных', 3, 4, 5, 6, 7, 8, 9, 10]